In [1]:
# The following code is coppied and adapted from the following webpage:
# https://www.geeksforgeeks.org/emotion-detection-using-bidirectional-lstm/

In [2]:
# Importing the required libraries
import keras
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Dense,Bidirectional
from nltk.tokenize import word_tokenize,sent_tokenize
from keras.layers import *
from sklearn.model_selection import cross_val_score 
import nltk
import pandas as pd
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\4K\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Previous code for import data

from py_isear.isear_loader import IsearLoader
attributes = ['EMOT','SIT']
target = ['TROPHO','TEMPER']
loader = IsearLoader(attributes, target, True)
data = loader.load_isear('isear.csv')

data.drop(data[data[1] == '[ No response.]'].index, inplace = True)
print(data.head())
#Hide it for now or the github page always takes forever to browse ;)
#data.get_data() # returns attributes
#data.get_target() # returns target
#data.get_freetext_content() # returns the text content of the database

In [3]:
df = pd.read_csv('text_emotion_isear_edit2.csv', header=1)
df

,EMOTION,SIT
0,joy,"During the period of falling in love, each tim..."
1,fear,When I was involved in a traffic accident.
2,anger,When I was driving home after several days of...
3,sadness,When I lost the person who meant the most to me.
4,disgust,The time I knocked a deer down - the sight of ...
...,...,...
7421,anger,Two years back someone invited me to be the tu...
7422,sadness,I had taken the responsibility to do something...
7423,disgust,I was at home and I heard a loud sound of spit...
7424,shame,I did not do the homework that the teacher had...


In [4]:
#Changed a bit as now we have a DataFrame
df.drop(df[df['SIT'] == '[ No response.]'].index, inplace = True)
df
#this does not work since the structure of the ISEAR file on the website is very different than the one we use, the one we use has a lot of additional data


,EMOTION,SIT
0,joy,"During the period of falling in love, each tim..."
1,fear,When I was involved in a traffic accident.
2,anger,When I was driving home after several days of...
3,sadness,When I lost the person who meant the most to me.
4,disgust,The time I knocked a deer down - the sight of ...
...,...,...
7421,anger,Two years back someone invited me to be the tu...
7422,sadness,I had taken the responsibility to do something...
7423,disgust,I was at home and I heard a loud sound of spit...
7424,shame,I did not do the homework that the teacher had...


In [5]:
#Take a look of the numbers of each emotion
print (df['EMOTION'].value_counts() )


EMOTION
fear       1074
sadness    1068
anger      1062
joy        1061
shame      1060
guilt      1056
disgust    1045
Name: count, dtype: int64


In [6]:
#let's get rid of anger and disgust to have a try
df = df[~df['EMOTION'].isin(['anger', 'disgust'])]
print (df['EMOTION'].value_counts() )

number_emotions = df['EMOTION'].unique()

EMOTION
fear       1074
sadness    1068
joy        1061
shame      1060
guilt      1056
Name: count, dtype: int64


#HIGHTLIGHT#HIGHTLIGHT#HIGHTLIGHT#HIGHTLIGHT#HIGHTLIGHT

In [7]:

#make all contexts (sentences) in dataset a list
texts =df["SIT"]
print(texts)




#HIGHTLIGHT
#Convert each text value to lowercase using the .str.lower() method
lower_texts = texts.str.lower()




print(lower_texts)
# Each  sentence in feel_arr is tokenized by the help of work tokenizer.
# If I have a sentence - 'I am happy'. 
# After word tokenizing it will convert into- ['I','am','happy']
lower_texts = [word_tokenize(sent) for sent in lower_texts]


#Now I added some codes to see how many words it has for the longest text
longest_text = max(lower_texts, key=len)
print (longest_text)
print("Sublist with the most elements:", len(longest_text))

0       During the period of falling in love, each tim...
1              When I was involved in a traffic accident.
3       When I lost the person who meant the most to me. 
5                         When I did not speak the truth.
6       When I caused problems for somebody because he...
                              ...                        
7419           I received a letter from a distant friend.
7420    My parents were out and I was the eldest at ho...
7422    I had taken the responsibility to do something...
7424    I did not do the homework that the teacher had...
7425    I had shouted at my younger brother and he was...
Name: SIT, Length: 5319, dtype: object
0       during the period of falling in love, each tim...
1              when i was involved in a traffic accident.
3       when i lost the person who meant the most to me. 
5                         when i did not speak the truth.
6       when i caused problems for somebody because he...
                              ...

Somehow we don't have the same dataset as our longest text has 200 words. <br>
for the next part I two choices: <br>
1. change the length for all sentences into 200, then it takes more time to train the model <br>
2. drop all texts more than certain words, 100 for instance.

I want to try the first way to see if its doable

In [8]:

# Defined a function padd in which each sentence length is fixed to 120.
# If length is less than 200 , then the word- '<padd>' is append
def padd(arr):
    for i in range(200-len(arr)):
        arr.append('<pad>')
    return arr[:200]
   
# call the padd function for each sentence in feel_arr
for i in range(len(lower_texts)):
    lower_texts[i]=padd(lower_texts[i])
 
print(lower_texts[0])

['during', 'the', 'period', 'of', 'falling', 'in', 'love', ',', 'each', 'time', 'that', 'we', 'met', 'and', 'ã¡', 'especially', 'when', 'we', 'had', 'not', 'met', 'for', 'a', 'long', 'time', '.', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<p

In [9]:

# Glove vector contains a 50 dimensional vector corresponding 
# to each word in dictionary.
vocab_f = 'glove.6B.50d.txt'
 
# embeddings_index is a dictionary which contains the mapping of
# word with its corresponding 50d vector.
embeddings_index = {}
with open(vocab_f, encoding='utf8') as f:
    for line in f:
        # splitting each line of the glove.6B.50d in a list of 
        # items- in which the first element is the word to be embedded,
        # and from second to the end of line contains the 50d vector.
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
         
# the embedding index of ','
embeddings_index[',']

array([ 0.013441,  0.23682 , -0.16899 ,  0.40951 ,  0.63812 ,  0.47709 ,
       -0.42852 , -0.55641 , -0.364   , -0.23938 ,  0.13001 , -0.063734,
       -0.39575 , -0.48162 ,  0.23291 ,  0.090201, -0.13324 ,  0.078639,
       -0.41634 , -0.15428 ,  0.10068 ,  0.48891 ,  0.31226 , -0.1252  ,
       -0.037512, -1.5179  ,  0.12612 , -0.02442 , -0.042961, -0.28351 ,
        3.5416  , -0.11956 , -0.014533, -0.1499  ,  0.21864 , -0.33412 ,
       -0.13872 ,  0.31806 ,  0.70358 ,  0.44858 , -0.080262,  0.63003 ,
        0.32111 , -0.46765 ,  0.22786 ,  0.36034 , -0.37818 , -0.56657 ,
        0.044691,  0.30392 ], dtype=float32)

In [10]:
# Embedding each word of the feel_arr
embedding_texts = []
# Iterate through each sub-list (sentences)
for word_list in lower_texts:
    # Initialize an empty list to store the embedding vectors for each word
    word_vectors = []
    for word in word_list:
        if word in embeddings_index:
            word_vector = embeddings_index[word]
            word_vectors.append(word_vector)
        else:
            # if the word to be embedded is '<padd>' append 0 fifty times
            word_vector = np.zeros(50, dtype='float32')
            word_vectors.append(word_vector)
    embedding_texts.append(word_vectors)

print(embedding_texts[0][0])

[ 0.29784   -0.018422  -0.71891   -0.4651    -0.45661   -0.0042153
 -0.74598    0.34662   -0.51781   -0.5877     0.18398   -0.36903
 -0.52225   -0.14082    0.83446   -0.26962   -0.89364   -0.11813
 -1.3076     0.475      0.52815   -0.021974   0.61869   -0.65362
 -0.14298   -1.6466    -0.05305   -0.17046    0.17048    0.75756
  3.5832     0.13775   -0.37811   -0.48736    0.0069906  0.59913
  0.31404    0.30734   -0.42397    0.35383   -0.97151    0.16082
 -0.63666   -0.20449   -0.070846  -0.32219   -0.049254  -0.41865
 -0.6899    -0.54908  ]


!!!!PROBLEM!!!!!<br>
I checked and found the the GloVe cannot recognize the capital letters. <br>
I can transfer all the capital letters into lowercase but the problem is for text message I personally uer capital letters a lot for emotion presentation<br>

----------------------------------------------------------------------------

Now the data should be ready (FINALLY) for training <br>
but still we need to coupling those random numbers with emotion (Input) <br>
Also we need to divide the data into training and testing sets. <br>
There are plenty of ways to divide, but to make it simple, I'll just make it 8:2 randomized.

Here is the Input, we can add more if we have more time.<br>
For instance, the length of the sentences

In [30]:
import tensorflow as tf
print (type(embedding_texts))

#embeeding_texts cannot be a list, for the website he made it NumPy-array
X = np.array(embedding_texts)
print (type(X))

<class 'list'>
<class 'numpy.ndarray'>


In [31]:
#One-Hot Encoding for all emotions, 
#instead of just give them numbers, by doing so we will have seven outputs

Y = pd.get_dummies(df['EMOTION'], prefix='emotion')
Y

,emotion_fear,emotion_guilt,emotion_joy,emotion_sadness,emotion_shame
0,False,False,True,False,False
1,True,False,False,False,False
3,False,False,False,True,False
5,False,False,False,False,True
6,False,True,False,False,False
...,...,...,...,...,...
7419,False,False,True,False,False
7420,True,False,False,False,False
7422,False,False,False,True,False
7424,False,False,False,False,True


In [32]:
from sklearn.model_selection import train_test_split
# Split into train and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.1,random_state=1)

I didn't check the distributions of emotions at the beginning. Now it just use the simpliest way to divide the dataset. <br>
If we get more time we should take a bit more time to analyize the datasets to choose a better way to divide. <br>
e.g. there are 40% of texts are all joy while only 5% of them are guilty <br>
Then the simple randomized distribution will make a horrible model

Now everything is ready to go and we can design the layers for the model <br>

-----------------------------------------------------------------------------

In [33]:
class BiLSTMModel:
    def __init__(self):
        self.model = Sequential()
        self.model.add(Bidirectional(LSTM(200, input_shape=(200, 50))))



        self.model.add(Dropout(0.2))
        
        self.model.add(Dense(5, activation='softmax'))
        self.model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
 
    def fit(self, X, Y, epochs, batch_size,verbose):
        self.model.fit(X, Y, epochs=epochs, batch_size=batch_size,verbose=verbose)
 
    def evaluate(self, X, Y, batch_size,verbose):
        return self.model.evaluate(X, Y, batch_size=batch_size,verbose=verbose)
 
    def predict(self, X):
        return self.model.predict(X)

In [34]:
from keras.models import load_model

# create an instance of the BiLSTMModel class
model = BiLSTMModel()

# fit the model on the input and target data
model.fit(X_train,Y_train, epochs=7, batch_size=16,verbose=1)


Epoch 1/7
 13/300 [>.............................] - ETA: 29s - loss: 1.6060 - accuracy: 0.2260

KeyboardInterrupt: 

Below is the code to test the best epochs, it will <br>
1. train the model for one epoch <br>
2. evaluation the test data with the testing model that it will get the accuracy and loss for testing dataset <br>
3. print the accuracy for both train and test

In [35]:
def compar(self, X_train, Y_train, X_test, Y_test, epochs, batch_size):
    train_accuracy_history = []  # For recording train accuracy
    test_accuracy_history = []   # For recording test accuracy

    for epoch in range(epochs):
        history = self.model.fit(X_train, Y_train, epochs=1, batch_size=batch_size, verbose=0)

        # Calculate training accuracy
        train_accuracy = history.history['accuracy'][0]
        train_accuracy_history.append(train_accuracy)


        test_loss, test_accuracy = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=0)
        test_accuracy_history.append(test_accuracy)

        print(f'Epoch {epoch + 1}/{epochs}' )
        print(f'Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}')

    return train_accuracy_history, test_accuracy_history


always reset the model with "model = BiLSTMModel()'<br>
or we are just trainning more cycle (epoch) from the previous model

In [37]:
model = BiLSTMModel()
compar(model,X_train, Y_train, X_test, Y_test, 15, 16)

KeyboardInterrupt: 

In [40]:
model = BiLSTMModel()
model.fit(X_train,Y_train, epochs=1, batch_size=16,verbose = 1)


300/300 [==============================] - 40s 127ms/step - loss: 1.4417 - accuracy: 0.3858


In [41]:
loss,accuracy = model.evaluate(X_test, Y_test, 16,verbose = 0)
print("Loss:", loss)
print("Accuracy:", accuracy)



Loss: 1.3211694955825806
Accuracy: 0.48308271169662476


Just addedd some code to save the model 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

CHANGE THE NAME YOU SAVE YOUR MODEL TO

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
#Uncomment this piece of code if you want to save the model 

#model.model.save('trained_model_Max_20231016_1.h5')  # creates a HDF5 file 'my_model.h5'
#del model  # deletes the existing model
    
# returns a compiled model
# identical to the previous one
#model = load_model('')

In [42]:
# Take a look of the model layers summarry
model.model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_12 (Bidirect  (None, 400)               401600    
 ional)                                                          
                                                                 
 dropout_12 (Dropout)        (None, 400)               0         
                                                                 
 dense_12 (Dense)            (None, 5)                 2005      
                                                                 
Total params: 403605 (1.54 MB)
Trainable params: 403605 (1.54 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [44]:
#This was just some example code on how to in theory make such a model, It won't be that relevant anymore
'''
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Sample training data
texts = ["This is a positive text.", "Negative sentiment here.", "I feel neutral about this."]
labels = [1, 0, 2]  # 1 for positive, 0 for negative, 2 for neutral

# Tokenize and pad the text data
tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
sequences = pad_sequences(sequences, maxlen=10, padding='post', truncating='post')
input_data_list = sequences.tolist()


# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=1000, output_dim=16, input_length=10))
model.add(LSTM(32))
model.add(Dense(3, activation='softmax'))  # Assuming you have 3 classes (positive, negative, neutral)

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(input_data_list, labels, epochs=10, verbose=2)

# Now, you can use the trained model to predict labels for new input text
new_text = ["I like this product."]
new_sequence = tokenizer.texts_to_sequences(new_text)
new_sequence = pad_sequences(new_sequence, maxlen=10, padding='post', truncating='post')
predictions = model.predict(new_sequence)

# Get the predicted label (argmax of the softmax output)
predicted_label = predictions.argmax()
print(f"Predicted label: {predicted_label}")
'''

'\nimport tensorflow as tf\nfrom tensorflow.keras.models import Sequential\nfrom tensorflow.keras.layers import Embedding, LSTM, Dense\nfrom tensorflow.keras.preprocessing.text import Tokenizer\nfrom tensorflow.keras.preprocessing.sequence import pad_sequences\n\n# Sample training data\ntexts = ["This is a positive text.", "Negative sentiment here.", "I feel neutral about this."]\nlabels = [1, 0, 2]  # 1 for positive, 0 for negative, 2 for neutral\n\n# Tokenize and pad the text data\ntokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")\ntokenizer.fit_on_texts(texts)\nsequences = tokenizer.texts_to_sequences(texts)\nsequences = pad_sequences(sequences, maxlen=10, padding=\'post\', truncating=\'post\')\ninput_data_list = sequences.tolist()\n\n\n# Build the LSTM model\nmodel = Sequential()\nmodel.add(Embedding(input_dim=1000, output_dim=16, input_length=10))\nmodel.add(LSTM(32))\nmodel.add(Dense(3, activation=\'softmax\'))  # Assuming you have 3 classes (positive, negative, neutral)\n